In [1]:
import rasterio
import pandas as pd 
import os 
import rasterio 
import numpy as np 
import pandas as pd 
import re

In [2]:
with rasterio.open('train/s2_Cocoa_ID_0bCYpY_2024_01.tif') as src:
    band_descriptions = [src.descriptions[i] if src.descriptions[i] else f'band_{i+1}' for i in range(src.count)]
    bands = [src.read(i + 1) for i in range(src.count)]
    transform = src.transform
    nodata = src.nodata

rows, cols = bands[0].shape
xs, ys = np.meshgrid(np.arange(cols), np.arange(rows))
xs, ys = rasterio.transform.xy(transform, ys, xs)
xs_flat = np.array(xs).flatten()
ys_flat = np.array(ys).flatten()

data = {'x': xs_flat, 'y': ys_flat}
for i, band in enumerate(bands):
    col_name = band_descriptions[i]
    data[col_name] = band.flatten()

# Create DataFrame
df = pd.DataFrame(data)

In [3]:
df.head()

,x,y,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12
0,-6.398116,7.407643,2404,2143,2050,1913,2193,3085,3515,3488,3872,1800,2935,2008
1,-6.398026,7.407643,2404,2146,2054,1916,2193,3085,3515,3437,3872,1800,2935,2008
2,-6.397936,7.407643,2404,2141,2045,1909,2157,3076,3455,3423,3847,1800,2915,1985
3,-6.397846,7.407643,2404,2144,2049,1904,2157,3076,3455,3467,3847,1800,2915,1985
4,-6.397757,7.407643,2404,2144,2049,1904,2157,3076,3455,3467,3847,1800,2915,1985


In [7]:
import os
import re
import pandas as pd
from collections import defaultdict

folder_path = './train'
pattern = re.compile(r"s2_(\w+)_ID_([a-zA-Z0-9]+)_(\d{4})_(\d{2})\.tif")

# Dictionary to track months per ID
id_months = defaultdict(set)
id_crops = {}

print(f"Scanning folder: {folder_path}...")

for filename in os.listdir(folder_path):
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        match = pattern.match(filename)
        if match:
            crop, crop_id, year, month = match.groups()
            id_months[crop_id].add(int(month))
            if crop_id not in id_crops:
                id_crops[crop_id] = crop  # store as int for easy comparison

# Summarize number of months per ID
summary = pd.DataFrame({
    'id': list(id_months.keys()),
    'crop': [id_crops[cid] for cid in id_months.keys()],
    'months': list(id_months.values())
})

summary['num_months'] = summary['months'].apply(len)

# Identify IDs with missing months
summary['missing_months'] = summary['months'].apply(lambda m: sorted(set(range(1, 13)) - set(m))) 

# Optional: Save report
# summary.sort_values('num_months').to_csv('missing_months_report.csv', index=False)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)  # Let pandas use unlimited width
pd.set_option('display.max_colwidth', None)

print(f"\nNumber of IDs with less than 12 months: {(summary['num_months'] < 12).sum()}")
print(summary[['id', 'crop','num_months', 'missing_months']])


Scanning folder: ./train...

Number of IDs with less than 12 months: 926
         id    crop  num_months    missing_months
0    0bCYpY   Cocoa           8      [5, 6, 8, 9]
1    0DNGt7   Cocoa           8      [5, 6, 8, 9]
2    0vYqno   Cocoa           8      [5, 6, 8, 9]
3    1AYcTD   Cocoa           8      [5, 6, 8, 9]
4    1dqP0R   Cocoa           8      [5, 6, 8, 9]
5    1n8WVN   Cocoa           8      [5, 6, 8, 9]
6    1yRRfH   Cocoa           9         [7, 8, 9]
7    2sZzMT   Cocoa           8      [5, 6, 8, 9]
8    2uKltK   Cocoa           8      [5, 6, 8, 9]
9    2wRuZz   Cocoa           8      [5, 6, 8, 9]
10   2zlkNI   Cocoa           8      [5, 6, 8, 9]
11   34ft12   Cocoa           8      [5, 6, 8, 9]
12   35EDH6   Cocoa           8      [5, 6, 8, 9]
13   3cdyNb   Cocoa           7  [6, 7, 8, 9, 10]
14   3oT458   Cocoa           8      [5, 6, 8, 9]
15   3ubhnR   Cocoa           8      [5, 6, 8, 9]
16   46GB1d   Cocoa           8      [5, 6, 8, 9]
17   50jFac   Cocoa        

In [11]:
total_missing_months = summary['missing_months'].apply(len).sum()
print(f"\n✅ Total missing months across all IDs: {total_missing_months}")


✅ Total missing months across all IDs: 4003


In [9]:
import os
import numpy as np 
import rasterio
from rasterio.enums import Resampling

input_dir = "./train"
output_dir = "./train_filled"
os.makedirs(output_dir, exist_ok=True)
year = 2024

def find_neighbors(existing, missing):
    lower = sorted([m for m in existing if m < missing])
    upper = sorted([m for m in existing if m > missing])
    return (lower[-1] if lower else None), (upper[0] if upper else None)

def average_rasters_rasterio(path1, path2, output_path):
    with rasterio.open(path1) as src1, rasterio.open(path2) as src2:
        profile = src1.profile.copy()
        bands = []

        for b in range(1, src1.count + 1):
            arr1 = src1.read(b).astype(np.float32)
            arr2 = src2.read(b).astype(np.float32)
            avg = (arr1 + arr2) / 2
            bands.append(avg)

        with rasterio.open(output_path, "w", **profile) as dst:
            for i, band in enumerate(bands, start=1):
                dst.write(band, i)
    return True


In [ ]:
for idx, row in summary.iterrows():
    crop_id = row['id'] 
    crop = row['crop']
    missing_months = row['missing_months']
    existing_months = sorted(set(range(1, 13)) - set(missing_months))

    for m in missing_months:
        prev, next_ = find_neighbors(existing_months, m)
        if prev is None or next_ is None:
            print(f"[{crop_id}] Skipping month {m}: no neighbors")
            continue 

        r1 = os.path.join(input_dir, f"s2_{crop}_ID_{crop_id}_{year}_{prev:02}.tif")
        r2 = os.path.join(input_dir, f"s2_{crop}_ID_{crop_id}_{year}_{next_:02}.tif")
        output = os.path.join(output_dir, f"s2_{crop}_ID_{crop_id}_{year}_{m:02}.tif")

        if not os.path.exists(r1) or not os.path.exists(r2):
            print(f"[{crop_id}] Missing raster(s) for month {m}: {r1} or {r2}")
            continue

        try:
            average_rasters_rasterio(r1, r2, output)
            print(f"[{crop_id}] Created missing month {m}")
        except Exception as e:
            print(f"[{crop_id}] Error processing month {m}: {e}")


[0bCYpY] Created missing month 5
[0bCYpY] Created missing month 6
[0bCYpY] Created missing month 8
[0bCYpY] Created missing month 9
[0DNGt7] Created missing month 5
[0DNGt7] Created missing month 6
[0DNGt7] Created missing month 8
[0DNGt7] Created missing month 9
[0vYqno] Created missing month 5
[0vYqno] Created missing month 6
[0vYqno] Created missing month 8
[0vYqno] Created missing month 9
[1AYcTD] Created missing month 5
[1AYcTD] Created missing month 6
[1AYcTD] Created missing month 8
[1AYcTD] Created missing month 9
[1dqP0R] Created missing month 5
[1dqP0R] Created missing month 6
[1dqP0R] Created missing month 8
[1dqP0R] Created missing month 9
[1n8WVN] Created missing month 5
[1n8WVN] Created missing month 6
[1n8WVN] Created missing month 8
[1n8WVN] Created missing month 9
[1yRRfH] Created missing month 7
[1yRRfH] Created missing month 8
[1yRRfH] Created missing month 9
[2sZzMT] Created missing month 5
[2sZzMT] Created missing month 6
[2sZzMT] Created missing month 8
[2sZzMT] C

### `Train data`

In [ ]:
# folder_path = './train'
# metadata_records = []

# # Regex pattern to parse filenames
# pattern = re.compile(r"s2_(\w+)_ID_([a-zA-Z0-9]+)_(\d{4})_(\d{2})\.tif")

# print(f"Scanning folder: {folder_path}...")
# for filename in os.listdir(folder_path):
#     if filename.endswith('.tif') or filename.endswith('.tiff'):
#         match = pattern.match(filename)
#         if match:
#             crop, crop_id, year, month = match.groups()
#             file_path = os.path.join(folder_path, filename)

#             try:
#                 with rasterio.open(file_path) as src:
#                     # Read all bands (assuming 12 bands: B1-B12)
#                     bands = src.read(range(1, 13))  # Shape: (12, height, width)
                    
#                     # Calculate mean reflectance per band (across all pixels)
#                     band_means = [np.mean(band) for band in bands]
                    
#                     # Create a record with band means as columns
#                     record = {
#                         'crop': crop,
#                         'id': crop_id,
#                         'year': int(year),
#                         'month': int(month),
#                         **{f'B{i+1}': mean_val for i, mean_val in enumerate(band_means)}
#                     }
                    
#                     metadata_records.append(record)
                    
#             except rasterio.errors.RasterioIOError as e:
#                 print(f"Warning: Could not open {file_path}. Skipping. Error: {e}")
#             except Exception as e:
#                 print(f"An unexpected error occurred with {file_path}. Skipping. Error: {e}")

# # Convert to DataFrame
# df_bands = pd.DataFrame(metadata_records)

# print(f"\nCollected data for {len(df_bands)} images.")
# print(f"Unique IDs found: {df_bands['id'].nunique()}")

Scanning folder: ./train...

Collected data for 7433 images.
Unique IDs found: 953


In [12]:
import os
import re
import numpy as np
import pandas as pd
import rasterio

metadata_records = []

# Folders to scan
folders = ['./train', './train_filled']

# Regex pattern to parse filenames
pattern = re.compile(r"s2_(\w+)_ID_([a-zA-Z0-9]+)_(\d{4})_(\d{2})\.tif")

for folder_path in folders: 
    print(f"Scanning folder: {folder_path}...")
    for filename in os.listdir(folder_path):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            match = pattern.match(filename)
            if match:
                crop, crop_id, year, month = match.groups()
                file_path = os.path.join(folder_path, filename)

                try:
                    with rasterio.open(file_path) as src:
                        bands = src.read(range(1, 13))  # 12 bands
                        band_means = [np.mean(band) for band in bands]

                        record = {
                            'crop': crop,
                            'id': crop_id,
                            'year': int(year),
                            'month': int(month),
                            **{f'B{i+1}': mean_val for i, mean_val in enumerate(band_means)}
                        }

                        metadata_records.append(record)

                except rasterio.errors.RasterioIOError as e:
                    print(f"Warning: Could not open {file_path}. Skipping. Error: {e}")
                except Exception as e:
                    print(f"An unexpected error occurred with {file_path}. Skipping. Error: {e}")

# Create final DataFrame
df_bands = pd.DataFrame(metadata_records)

print(f"\n✅ Collected data for {len(df_bands)} images.")
print(f"📌 Unique IDs found: {df_bands['id'].nunique()}")
print(f"📆 Months covered (min-max): {df_bands['month'].min()} – {df_bands['month'].max()}")


Scanning folder: ./train...
Scanning folder: ./train_filled...

✅ Collected data for 11436 images.
📌 Unique IDs found: 953
📆 Months covered (min-max): 1 – 12


In [16]:
df_bands.head()

,crop,id,year,month,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12
0,Cocoa,0bCYpY,2024,1,2406.538639,2160.567474,2090.970012,1931.701269,2248.763937,3324.108804,3814.095348,3768.468281,4191.387543,1864.017301,3045.683968,2070.279892
1,Cocoa,0bCYpY,2024,2,2596.987697,2359.017301,2289.663591,2198.013072,2448.089965,3377.813533,3882.286044,3741.199923,4345.721261,1431.754325,3454.167243,2253.949635
2,Cocoa,0bCYpY,2024,3,2361.674740,2089.342176,2061.183775,1714.469050,2142.346021,3712.004998,4379.708958,4167.754325,4812.191465,1420.314494,2925.771626,1819.683968
3,Cocoa,0bCYpY,2024,4,2404.282968,2141.820069,2129.459439,1803.307574,2210.539408,3842.402537,4566.769704,4352.041907,5046.622837,1415.896963,3201.394079,1990.712803
4,Cocoa,0bCYpY,2024,7,3437.642445,3193.304114,3102.590927,2967.977701,3169.578239,4420.143406,5096.212611,4804.905805,5499.835832,1595.154556,4038.272972,2964.091888


In [17]:
len(df_bands)

11436

In [14]:
# Count unique months per ID
month_counts = df_bands.groupby('id')['month'].nunique().sort_values(ascending=False).reset_index()
month_counts.columns = ['id', 'num_months']

# Filter IDs with fewer than 12 months
incomplete_ids = month_counts[month_counts['num_months'] < 12]

# Display the result
print(incomplete_ids)

# Count how many IDs have less than 12 months
print(f"Number of IDs with less than 12 months: {len(incomplete_ids)}") 


Empty DataFrame
Columns: [id, num_months]
Index: []
Number of IDs with less than 12 months: 0


In [ ]:
df_bands['month'] = df_bands['month'].astype(int)

# List of all months
all_months = list(range(1, 13))

id_info = df_bands[['id', 'crop', 'year']].drop_duplicates().reset_index(drop=True)

# Create full month grid for each id
full_index = (
    id_info
    .merge(pd.DataFrame({'month': all_months}), how='cross')
    .sort_values(['id', 'month'])
) 


In [20]:
merged = (
    full_index
    .merge(df_bands, on=['id', 'crop', 'year', 'month'], how='left')
    .sort_values(['id', 'month'])
)

# Melt to long format for band values
melted = merged.melt(
    id_vars=['id', 'crop', 'year', 'month'],
    value_vars=[f'B{i}' for i in range(1, 13)],
    var_name='band',
    value_name='mean_reflectance'
) 

# Create a combined band_month column, e.g., B1_01
melted['band_month'] = melted['band'] + '_' + melted['month'].astype(str).str.zfill(2)

# Pivot wide
wide_df = (
    melted
    .pivot(index=['id', 'crop', 'year'], columns='band_month', values='mean_reflectance')
    .reset_index()
)


In [21]:
wide_df.head()

band_month,id,crop,year,B10_01,B10_02,B10_03,B10_04,B10_05,B10_06,B10_07,B10_08,B10_09,B10_10,B10_11,B10_12,B11_01,B11_02,B11_03,B11_04,B11_05,B11_06,B11_07,B11_08,B11_09,B11_10,B11_11,B11_12,B12_01,B12_02,B12_03,B12_04,B12_05,B12_06,B12_07,B12_08,B12_09,B12_10,B12_11,B12_12,B1_01,B1_02,B1_03,B1_04,B1_05,B1_06,B1_07,B1_08,B1_09,B1_10,B1_11,B1_12,B2_01,B2_02,B2_03,B2_04,B2_05,B2_06,B2_07,B2_08,B2_09,B2_10,B2_11,B2_12,B3_01,B3_02,B3_03,B3_04,B3_05,B3_06,B3_07,B3_08,B3_09,B3_10,B3_11,B3_12,B4_01,B4_02,B4_03,B4_04,B4_05,B4_06,B4_07,B4_08,B4_09,B4_10,B4_11,B4_12,B5_01,B5_02,B5_03,B5_04,B5_05,B5_06,B5_07,B5_08,B5_09,B5_10,B5_11,B5_12,B6_01,B6_02,B6_03,B6_04,B6_05,B6_06,B6_07,B6_08,B6_09,B6_10,B6_11,B6_12,B7_01,B7_02,B7_03,B7_04,B7_05,B7_06,B7_07,B7_08,B7_09,B7_10,B7_11,B7_12,B8_01,B8_02,B8_03,B8_04,B8_05,B8_06,B8_07,B8_08,B8_09,B8_10,B8_11,B8_12,B9_01,B9_02,B9_03,B9_04,B9_05,B9_06,B9_07,B9_08,B9_09,B9_10,B9_11,B9_12
0,059i1w,Palm,2024,1474.543637,1239.377163,1542.324491,1379.364860,1536.788158,1536.788158,1694.727413,1494.621300,1494.621300,1494.621300,1294.968474,1607.919646,2801.226451,3028.440984,4400.201461,2695.100346,3332.707036,3332.707036,3970.819685,3021.181853,3021.181853,3021.181853,2072.050750,2779.179546,1853.092272,1990.196463,3083.952326,2055.690119,2383.211457,2383.211457,2711.229143,2056.236448,2056.236448,2056.236448,1401.740869,1850.972703,2351.206459,2527.040754,3402.844291,2854.747020,3002.138024,3002.138024,3149.905421,2743.819300,2743.819300,2743.819300,2338.245290,2401.714725,2109.881584,2258.189927,3104.113802,2606.915417,2797.082276,2797.082276,2987.752018,2503.614379,2503.614379,2503.614379,2019.973087,2156.301807,1990.108420,2126.188774,3088.089196,2440.797001,2655.868512,2655.868512,2871.420992,2346.659362,2346.659362,2346.659362,1822.412918,2027.460592,1786.064591,1951.783929,3021.868128,2265.879662,2479.364860,2479.364860,2693.356017,2116.516340,2116.516340,2116.516340,1540.176086,1823.204921,1987.347943,2118.951557,3222.898501,2354.412534,2640.525567,2640.525567,2927.125336,2309.267974,2309.267974,2309.267974,1691.911957,2030.707036,3057.400615,2950.051903,4590.279508,3115.645521,3856.492503,3856.492503,4597.837755,3637.554402,3637.554402,3637.554402,2677.743176,3113.811226,3552.858131,3456.313725,5324.066128,3619.833526,4563.402537,4563.402537,5507.506728,4361.195309,4361.195309,4361.195309,3215.420223,3624.427912,3459.990773,3260.473664,4923.105729,3371.670896,4317.015379,4317.015379,5262.844675,4155.601692,4155.601692,4155.601692,3048.864283,3537.839677,3887.469050,3798.426759,5751.903883,3814.747020,4905.570934,4905.570934,5996.912341,4718.268743,4718.268743,4718.268743,3440.110342,3934.848904
1,0612VR,Palm,2024,1461.167243,1219.619377,1352.573626,1435.990773,1381.933872,1381.933872,1328.377932,1385.110342,1385.110342,1385.110342,1442.457132,1647.507113,2846.038831,2468.993464,3015.137255,3162.384852,2663.911957,2663.911957,2165.951942,2479.250288,2479.250288,2479.250288,2793.036140,2801.091119,1914.185313,1800.263745,1893.530565,2081.709727,1773.384852,1773.384852,1465.571703,1633.097655,1633.097655,1633.097655,1801.120723,1866.706651,2348.718954,2799.190696,2361.294118,2597.070742,2481.851596,2481.851596,2367.109189,2410.880815,2410.880815,2410.880815,2455.214917,2394.076125,2105.990388,2540.216455,2090.968474,2360.016532,2212.038447,2212.038447,2064.560938,2105.400231,2105.400231,2105.400231,2146.759323,2138.321799,1978.349481,2346.912726,1997.559016,2257.769319,2067.003845,2067.003845,1876.728566,1942.715879,1942.715879,1942.715879,2009.186082,2008.248750,1809.209535,2174.666282,1759.677432,2023.410996,1816.218762,1816.218762,1609.520569,1672.879277,1672.879277,1672.879277,1736.723183,1808.249904,2005.582468,2206.696655,2046.580161,2281.309881,2034.438293,2034.438293,1788.063053,1893.937332,1893.937332,1893.937332,2000.304498,2038.121876,2865.631296,2746.864667,3406.793541,3604.465590,3198.617839,3198.617839,2793.291042,3049.181084,3049.181084,3049.181084,3305.602076,3059.707036,3316.304498,31

In [22]:
wide_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Columns: 147 entries, id to B9_12
dtypes: float64(144), int64(1), object(2)
memory usage: 1.1+ MB


In [23]:
wide_df.isna().sum()

band_month
id        0
crop      0
year      0
B10_01    0
B10_02    0
B10_03    0
B10_04    0
B10_05    0
B10_06    0
B10_07    0
B10_08    0
B10_09    0
B10_10    0
B10_11    0
B10_12    0
B11_01    0
B11_02    0
B11_03    0
B11_04    0
B11_05    0
B11_06    0
B11_07    0
B11_08    0
B11_09    0
B11_10    0
B11_11    0
B11_12    0
B12_01    0
B12_02    0
B12_03    0
B12_04    0
B12_05    0
B12_06    0
B12_07    0
B12_08    0
B12_09    0
B12_10    0
B12_11    0
B12_12    0
B1_01     0
B1_02     0
B1_03     0
B1_04     0
B1_05     0
B1_06     0
B1_07     0
B1_08     0
B1_09     0
B1_10     0
B1_11     0
B1_12     0
B2_01     0
B2_02     0
B2_03     0
B2_04     0
B2_05     0
B2_06     0
B2_07     0
B2_08     0
B2_09     0
B2_10     0
B2_11     0
B2_12     0
B3_01     0
B3_02     0
B3_03     0
B3_04     0
B3_05     0
B3_06     0
B3_07     0
B3_08     0
B3_09     0
B3_10     0
B3_11     0
B3_12     0
B4_01     0
B4_02     0
B4_03     0
B4_04     0
B4_05     0
B4_06     0
B4_07     0
B4_08

In [25]:
# wide_df.drop(columns=["band_month"], inplace= True)
wide_df.to_csv('Rasterio_Train.csv')

### `Test data processing`

In [26]:
folder_path = './test'
pattern = re.compile(r"s2_(\w+)_ID_([a-zA-Z0-9]+)_(\d{4})_(\d{2})\.tif")

# Dictionary to track months per ID
id_months = defaultdict(set)
id_crops = {}

print(f"Scanning folder: {folder_path}...")

for filename in os.listdir(folder_path):
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        match = pattern.match(filename)
        if match:
            crop, crop_id, year, month = match.groups()
            id_months[crop_id].add(int(month))
            if crop_id not in id_crops:
                id_crops[crop_id] = crop  # store as int for easy comparison

# Summarize number of months per ID
summary2 = pd.DataFrame({
    'id': list(id_months.keys()),
    'crop': [id_crops[cid] for cid in id_months.keys()],
    'months': list(id_months.values())
})

summary2['num_months'] = summary2['months'].apply(len)

# Identify IDs with missing months
summary2['missing_months'] = summary2['months'].apply(lambda m: sorted(set(range(1, 13)) - set(m))) 

# Optional: Save report
# summary.sort_values('num_months').to_csv('missing_months_report.csv', index=False)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)  # Let pandas use unlimited width 
pd.set_option('display.max_colwidth', None)

print(f"\nNumber of IDs with less than 12 months: {(summary2['num_months'] < 12).sum()}")
print(summary2[['id', 'crop','num_months', 'missing_months']])


Scanning folder: ./test...

Number of IDs with less than 12 months: 270
         id     crop  num_months    missing_months
0    000167  Unknown           7  [5, 6, 8, 9, 10]
1    004157  Unknown          10            [5, 6]
2    010554  Unknown           8     [6, 8, 9, 10]
3    016218  Unknown           7  [5, 6, 8, 9, 10]
4    018928  Unknown           8      [5, 6, 8, 9]
5    019141  Unknown           7  [5, 6, 8, 9, 10]
6    026221  Unknown           8     [5, 8, 9, 10]
7    029679  Unknown           8     [6, 8, 9, 10]
8    032593  Unknown           8     [6, 8, 9, 10]
9    034755  Unknown          10            [5, 6]
10   039466  Unknown          11               [6]
11   043624  Unknown           8     [6, 8, 9, 10]
12   043761  Unknown           8     [5, 8, 9, 10]
13   046027  Unknown           7  [5, 6, 8, 9, 10]
14   046063  Unknown           8     [6, 8, 9, 10]
15   051942  Unknown           7  [5, 6, 8, 9, 10]
16   052303  Unknown           7  [5, 6, 8, 9, 10]
17   05879

In [ ]:
total_missing_months2 = summary2['missing_months'].apply(len).sum() 
print(f"\n✅ Total missing months across all IDs: {total_missing_months2}") 


✅ Total missing months across all IDs: 1183


In [30]:
test_dir = "./test"
output_test_dir = "./test_filled"
os.makedirs(output_test_dir, exist_ok=True) 
year = 2024

import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio.warp import reproject

def average_rasters_multiband(raster_path1, raster_path2, output_path):
    with rasterio.open(raster_path1) as src1, rasterio.open(raster_path2) as src2:
        if src1.count != 12 or src2.count != 12:
            print(f"Expected 12 bands, got {src1.count} and {src2.count}")
            return False

        # Get shapes
        shape1 = src1.read(1).shape
        shape2 = src2.read(1).shape
        area1 = shape1[0] * shape1[1]
        area2 = shape2[0] * shape2[1]

        # Choose reference raster
        if area1 >= area2:
            ref_src, other_src = src1, src2
        else:
            ref_src, other_src = src2, src1

        meta = ref_src.meta.copy()
        meta.update({
            "driver": "GTiff",
            "count": 12,
            "compress": "deflate"
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            for i in range(1, 13):
                ref_band = ref_src.read(i).astype(np.float32)
                other_band = other_src.read(i).astype(np.float32)

                # Resample if shapes differ
                if ref_band.shape != other_band.shape:
                    resampled = np.empty_like(ref_band)
                    reproject(
                        source=other_band,
                        destination=resampled,
                        src_transform=other_src.transform,
                        src_crs=other_src.crs,
                        dst_transform=ref_src.transform,
                        dst_crs=ref_src.crs,
                        resampling=Resampling.bilinear
                    )
                    other_band = resampled

                avg = (ref_band + other_band) / 2.0
                dst.write(avg, i)

    return True


the test data has some differences in the resolution for som emonths and so te above code helps to fix that by choosing the best resolution so that generating new raster files can be much easier without any errors. 

In [31]:
for idx, row in summary2.iterrows():
    crop_id = row['id'] 
    crop = row['crop']
    missing_months = row['missing_months']
    existing_months = sorted(set(range(1, 13)) - set(missing_months))

    for m in missing_months:
        prev, next_ = find_neighbors(existing_months, m)
        if prev is None or next_ is None: 
            print(f"[{crop_id}] Skipping month {m}: no neighbors")
            continue 

        r1 = os.path.join(test_dir, f"s2_{crop}_ID_{crop_id}_{year}_{prev:02}.tif")
        r2 = os.path.join(test_dir, f"s2_{crop}_ID_{crop_id}_{year}_{next_:02}.tif")
        output = os.path.join(output_test_dir, f"s2_{crop}_ID_{crop_id}_{year}_{m:02}.tif")

        if not os.path.exists(r1) or not os.path.exists(r2):
            print(f"[{crop_id}] Missing raster(s) for month {m}: {r1} or {r2}")
            continue

        try:
            average_rasters_multiband(r1, r2, output)
            print(f"[{crop_id}] Created missing month {m}")
        except Exception as e:
            print(f"[{crop_id}] Error processing month {m}: {e}")


[000167] Created missing month 5
[000167] Created missing month 6
[000167] Created missing month 8
[000167] Created missing month 9
[000167] Created missing month 10
[004157] Created missing month 5
[004157] Created missing month 6
[010554] Created missing month 6
[010554] Created missing month 8
[010554] Created missing month 9
[010554] Created missing month 10
[016218] Created missing month 5
[016218] Created missing month 6
[016218] Created missing month 8
[016218] Created missing month 9
[016218] Created missing month 10
[018928] Created missing month 5
[018928] Created missing month 6
[018928] Created missing month 8
[018928] Created missing month 9
[019141] Created missing month 5
[019141] Created missing month 6
[019141] Created missing month 8
[019141] Created missing month 9
[019141] Created missing month 10
[026221] Created missing month 5
[026221] Created missing month 8
[026221] Created missing month 9
[026221] Created missing month 10
[029679] Created missing month 6
[0296

In [32]:
metadata_records2 = []

# Folders to scan
folders = ['./test', './test_filled']

# Regex pattern to parse filenames
pattern = re.compile(r"s2_(\w+)_ID_([a-zA-Z0-9]+)_(\d{4})_(\d{2})\.tif")

for folder_path in folders: 
    print(f"Scanning folder: {folder_path}...")
    for filename in os.listdir(folder_path):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            match = pattern.match(filename)
            if match:
                crop, crop_id, year, month = match.groups()
                file_path = os.path.join(folder_path, filename)

                try:
                    with rasterio.open(file_path) as src:
                        bands = src.read(range(1, 13))  # 12 bands
                        band_means = [np.mean(band) for band in bands]

                        record = {
                            'crop': crop,
                            'id': crop_id,
                            'year': int(year),
                            'month': int(month),
                            **{f'B{i+1}': mean_val for i, mean_val in enumerate(band_means)}
                        }

                        metadata_records2.append(record)

                except rasterio.errors.RasterioIOError as e:
                    print(f"Warning: Could not open {file_path}. Skipping. Error: {e}")
                except Exception as e:
                    print(f"An unexpected error occurred with {file_path}. Skipping. Error: {e}")

# Create final DataFrame
df_test = pd.DataFrame(metadata_records2)

print(f"\n✅ Collected data for {len(df_test)} images.")
print(f"📌 Unique IDs found: {df_test['id'].nunique()}")
print(f"📆 Months covered (min-max): {df_test['month'].min()} – {df_test['month'].max()}")


Scanning folder: ./test...
Scanning folder: ./test_filled...

✅ Collected data for 3384 images.
📌 Unique IDs found: 282
📆 Months covered (min-max): 1 – 12


In [33]:
len(df_test)

3384

In [34]:
df_test.head()

,crop,id,year,month,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12
0,Unknown,000167,2024,1,2609.715879,2319.438293,2164.212995,1925.427143,2148.793156,3227.035371,3749.058055,3572.120338,4075.692426,1413.550173,2855.518262,1886.657824
1,Unknown,000167,2024,2,2879.517878,2623.550557,2415.243368,2150.736640,2289.144175,3166.971165,3617.379854,3418.487505,3847.446751,1331.888889,2539.895425,1768.923875
2,Unknown,000167,2024,3,2498.327951,2198.668589,2122.976547,1806.127259,2180.645905,3973.881200,4830.557093,4538.913110,5271.803153,1412.000769,3303.197616,2003.978854
3,Unknown,000167,2024,4,2412.658593,2092.028066,2016.951942,1650.023453,2074.529412,4015.772011,4936.122261,4618.902730,5407.549020,1397.169166,3175.590158,1849.454441
4,Unknown,000167,2024,7,3982.745098,3830.524798,3743.940408,3867.745098,3970.679739,4742.793156,5225.122261,4903.528643,5513.909650,1891.722030,4415.630527,3571.981930


In [35]:
df_test['month'] = df_test['month'].astype(int)

# List of all months
all_months = list(range(1, 13))

id_info = df_test[['id', 'crop', 'year']].drop_duplicates().reset_index(drop=True)

# Create full month grid for each id
full_index = (
    id_info
    .merge(pd.DataFrame({'month': all_months}), how='cross')
    .sort_values(['id', 'month'])
) 


In [37]:
merged = (
    full_index
    .merge(df_test, on=['id', 'crop', 'year', 'month'], how='left')
    .sort_values(['id', 'month'])
)

# Melt to long format for band values
melted = merged.melt(
    id_vars=['id', 'crop', 'year', 'month'],
    value_vars=[f'B{i}' for i in range(1, 13)],
    var_name='band',
    value_name='mean_reflectance'
) 

# Create a combined band_month column, e.g., B1_01
melted['band_month'] = melted['band'] + '_' + melted['month'].astype(str).str.zfill(2)

# Pivot wide 
wide_df_test = (
    melted
    .pivot(index=['id', 'crop', 'year'], columns='band_month', values='mean_reflectance')
    .reset_index()
)


In [38]:
wide_df_test.head()

band_month,id,crop,year,B10_01,B10_02,B10_03,B10_04,B10_05,B10_06,B10_07,B10_08,B10_09,B10_10,B10_11,B10_12,B11_01,B11_02,B11_03,B11_04,B11_05,B11_06,B11_07,B11_08,B11_09,B11_10,B11_11,B11_12,B12_01,B12_02,B12_03,B12_04,B12_05,B12_06,B12_07,B12_08,B12_09,B12_10,B12_11,B12_12,B1_01,B1_02,B1_03,B1_04,B1_05,B1_06,B1_07,B1_08,B1_09,B1_10,B1_11,B1_12,B2_01,B2_02,B2_03,B2_04,B2_05,B2_06,B2_07,B2_08,B2_09,B2_10,B2_11,B2_12,B3_01,B3_02,B3_03,B3_04,B3_05,B3_06,B3_07,B3_08,B3_09,B3_10,B3_11,B3_12,B4_01,B4_02,B4_03,B4_04,B4_05,B4_06,B4_07,B4_08,B4_09,B4_10,B4_11,B4_12,B5_01,B5_02,B5_03,B5_04,B5_05,B5_06,B5_07,B5_08,B5_09,B5_10,B5_11,B5_12,B6_01,B6_02,B6_03,B6_04,B6_05,B6_06,B6_07,B6_08,B6_09,B6_10,B6_11,B6_12,B7_01,B7_02,B7_03,B7_04,B7_05,B7_06,B7_07,B7_08,B7_09,B7_10,B7_11,B7_12,B8_01,B8_02,B8_03,B8_04,B8_05,B8_06,B8_07,B8_08,B8_09,B8_10,B8_11,B8_12,B9_01,B9_02,B9_03,B9_04,B9_05,B9_06,B9_07,B9_08,B9_09,B9_10,B9_11,B9_12
0,000167,Unknown,2024,1413.550173,1331.888889,1412.000769,1397.169166,1644.213379,1644.213379,1891.722030,1708.497116,1708.497116,1708.497116,1525.732026,1433.654364,2855.518262,2539.895425,3303.197616,3175.590158,3795.352941,3795.352941,4415.630527,3899.907343,3899.907343,3899.907343,3384.682814,3169.801999,1886.657824,1768.923875,2003.978854,1849.454441,2710.461361,2710.461361,3571.981930,2991.658593,2991.658593,2991.658593,2411.835448,2325.581315,2609.715879,2879.517878,2498.327951,2412.658593,3197.414456,3197.414456,3982.745098,4096.858131,4096.858131,4096.858131,4211.452518,2963.590542,2319.438293,2623.550557,2198.668589,2092.028066,2961.026913,2961.026913,3830.524798,3973.384852,3973.384852,3973.384852,4116.742407,2725.757401,2164.212995,2415.243368,2122.976547,2016.951942,2880.200692,2880.200692,3743.940408,3799.025759,3799.025759,3799.025759,3854.615148,2545.127259,1925.427143,2150.736640,1806.127259,1650.023453,2758.629373,2758.629373,3867.745098,3822.941945,3822.941945,3822.941945,3778.637063,2397.240292,2148.793156,2289.144175,2180.645905,2074.529412,3022.333333,3022.333333,3970.679739,3838.797001,3838.797001,3838.797001,3707.420992,2493.164937,3227.035371,3166.971165,3973.881200,4015.772011,4379.045367,4379.045367,4742.793156,4526.409458,4526.409458,4526.409458,4310.507497,3315.565552,3749.058055,3617.379854,4830.557093,4936.122261,5080.386390,5080.386390,5225.122261,5015.001153,5015.001153,5015.001153,4805.375625,3798.177240,3572.120338,3418.487505,4538.913110,4618.902730,4760.963860,4760.963860,4903.528643,4716.072280,4716.072280,4716.072280,4529.125336,3620.015379,4075.692426,3847.446751,5271.803153,5407.549020,5460.485967,5460.485967,5513.909650,5239.878893,5239.878893,5239.878893,4966.336794,4038.545944
1,004157,Unknown,2024,1415.679354,2062.143791,1200.608997,3750.434448,2787.703960,2787.703960,1825.451365,1872.845060,1998.124567,2539.268743,1397.011150,1540.015763,2662.200308,4933.703191,2139.427528,5799.198770,5456.305652,5456.305652,5113.921184,5441.409458,5757.372549,4718.469819,2737.538639,2624.426759,1769.020377,3699.992311,1526.786621,4097.788927,3977.247982,3977.247982,3857.209150,4027.468281,3939.607459,3222.975394,1722.597847,1832.093426,2360.822760,4489.122261,2376.112265,8690.462130,7160.179162,7160.179162,5630.387928,5942.344098,6845.001153,6892.495194,2348.886967,2478.886198,2120.824683,4609.047674,2080.966551,8783.951942,7107.382545,7107.382545,5431.305652,5837.670896,6687.438293,6698.999231,2043.192618,2272.605536,1992.705882,4515.159170,1900.024606,8567.352941,6891.337178,6891.337178,5215.828143,5657.994233,6489.687812,6394.071126,1940.464052,2123.366398,1764.362553,4571.098424,1673.139562,8965.561323,7160.378316,7160.378316,5355.702038,5893.863899,6751.317186,6571.015763,1605.101884,1896.111111,1993.462130,4615.126874,1790.698962,8816.742407,7004.271434,7004.271434,5192.300654,5642.050750,6545.121492,6435.380238,1937.432141,2099.395617,2977.084198,5354.599000,2440.800846,9038.602461,7382.264129,7382.264129,5726.434448,6071.156478,7075.429066,6787.313341,3427.830065,3055.274510,3457.005

In [39]:
wide_df_test.to_csv("Raster_Test.csv")

### `method2` 

In [ ]:
# Ensure month is integer
df_bands['month'] = df_bands['month'].astype(int)

# List of all months
all_months = list(range(1, 13))

# Unique (id, crop, year) combinations
id_info = df_bands[['id', 'crop', 'year']].drop_duplicates()

# Create full month grid for each id
full_index = (
    id_info
    .merge(pd.DataFrame({'month': all_months}), how='cross')  # all combinations
    .sort_values(['id', 'month'])
)

# Merge to get full data with NaNs for missing months
df_full = full_index.merge(df_bands, on=['id', 'crop', 'year', 'month'], how='left')

# Interpolate band columns per ID
band_cols = ['B1','B2', 'B3', 'B4', 'B5','B6','B7','B8','B8A','B9', 'B11', 'B12']
df_full[band_cols] = (
    df_full
    .sort_values(['id', 'month'])
    .groupby('id')[band_cols]
    .transform(lambda group: group.interpolate(method='linear', limit_direction='both'))
)

# Optional: check that now all IDs have 12 months
assert df_full.groupby('id')['month'].nunique().eq(12).all()

# Preview result
df_full


,id,crop,year,month,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12
0,059i1w,Palm,2024,1,2351.206459,2109.881584,1990.108420,1786.064591,1987.347943,3057.400615,3552.858131,3459.990773,3887.469050,1474.543637,2801.226451,1853.092272
1,059i1w,Palm,2024,2,2527.040754,2258.189927,2126.188774,1951.783929,2118.951557,2950.051903,3456.313725,3260.473664,3798.426759,1239.377163,3028.440984,1990.196463
2,059i1w,Palm,2024,3,3402.844291,3104.113802,3088.089196,3021.868128,3222.898501,4590.279508,5324.066128,4923.105729,5751.903883,1542.324491,4400.201461,3083.952326
3,059i1w,Palm,2024,4,2854.747020,2606.915417,2440.797001,2265.879662,2354.412534,3115.645521,3619.833526,3371.670896,3814.747020,1379.364860,2695.100346,2055.690119
4,059i1w,Palm,2024,5,2953.133154,2733.860951,2584.338331,2408.371780,2545.316801,3609.709599,4249.057926,4002.062156,4542.135461,1484.485711,3120.340126,2274.203127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11431,zzXzSQ,Cocoa,2024,8,5415.801871,5301.431501,5269.531847,5461.757017,5514.082661,6501.959759,7168.708317,6675.902986,7540.333718,2092.932590,5361.966679,3963.301166
11432,zzXzSQ,Cocoa,2024,9,3899.074715,3681.881071,3626.566192,3532.808151,3750.489043,5175.958349,5992.830706,5619.699859,6413.459439,1741.213892,4169.932590,2862.000256
11433,zzXzSQ,Cocoa,2024,10,2382.347559,2062.330642,1983.600538,1603.859285,1986.895425,3849.956940,4816.953095,4563.496732,5286.585160,1389.495194,2977.898501,1760.699346
11434,zzXzSQ,Cocoa,2024,11,2687.042291,2378.164937,2233.518262,1951.087274,2224.287966,3519.809689,4183.843137,4063.547482,4566.995002,1577.669742,2862.173010,1901.473664


In [3]:
df_full

,id,crop,year,month,B2,B3,B4,B8,B11,B12
0,059i1w,Palm,2024,1,2109.722991,1989.978470,1785.593618,3461.636486,2800.443483,1852.282968
1,059i1w,Palm,2024,2,3562.627067,3371.984237,3263.181084,4193.700500,3018.953864,2377.037293
2,059i1w,Palm,2024,3,3131.000000,3115.302576,3055.247982,4933.824298,4427.715110,3110.535948
3,059i1w,Palm,2024,4,2996.813413,2958.713019,2880.184905,4627.310602,4109.492743,2907.025062
4,059i1w,Palm,2024,5,2862.626826,2802.123462,2705.121828,4320.796905,3791.270377,2703.514177
...,...,...,...,...,...,...,...,...,...,...
11431,zzXzSQ,Cocoa,2024,8,2135.966383,2094.358492,1793.582252,4422.691200,3092.618296,1956.551230
11432,zzXzSQ,Cocoa,2024,9,2142.036669,2100.906046,1796.512351,4494.683151,3085.550990,1956.970973
11433,zzXzSQ,Cocoa,2024,10,2148.106954,2107.453600,1799.442450,4566.675101,3078.483684,1957.390715
11434,zzXzSQ,Cocoa,2024,11,2154.177240,2114.001153,1802.372549,4638.667051,3071.416378,1957.810458


Method Used: Linear Interpolation (method='linear') with limit_direction='both', followed by ffill() (forward fill) and bfill() (backward fill).
Rationale:

Gradual Changes: Band reflectance values for a specific crop location are expected to change gradually from month to month. This reflects continuous processes such as crop growth, senescence, and seasonal environmental shifts (e.g., changes in soil moisture). Linear interpolation assumes a steady rate of change between known data points, which aligns well with these natural, continuous biological and physical phenomena. This approach avoids introducing unrealistic, abrupt jumps in the time series.
Physical Basis: Satellite reflectance data, particularly in the spectral bands used, typically exhibits smooth transitions over time. The interpolated values will always fall within the range of the bounding observed values, preventing biologically or physically impossible extrapolations.

Addressing Edge Cases (ffill() and bfill()): Even with limit_direction='both', NaN values can persist at the very beginning or end of a time series for a particular (id, crop, year) group if no valid data points are available before or after the missing period. Applying ffill() and bfill() subsequently ensures that these leading or trailing NaNs are filled by propagating the nearest available valid observation.
Aim to Maintain True Values: By interpolating and filling based on neighboring temporal data points, we aim to reconstruct the most probable reflectance values given the observed trends. This approach is critical for preserving the unique phenological (seasonal growth) signature of each individual crop location throughout the year.

In [16]:
# Ensure month is integer
df['month'] = df['month'].astype(int)

# List of all months
all_months = list(range(1, 13))

id_info = df[['id', 'crop', 'year']].drop_duplicates().reset_index(drop=True)

# Create full month grid for each id
full_index = (
    id_info
    .merge(pd.DataFrame({'month': all_months}), how='cross')
    .sort_values(['id', 'month'])
)

# Merge to get full data with NaNs for missing months
df_full = full_index.merge(df, on=['id', 'crop', 'year', 'month'], how='left')

# Interpolate band columns per ID
band_cols = ['B1','B2', 'B3', 'B4', 'B5','B6','B7','B8','B8A','B9', 'B11', 'B12']
# df_full[band_cols] = (
#     df_full
#     .sort_values(['id', 'month'])
#     .groupby(['id', 'crop', 'year'])[band_cols] # Group also by year and crop for interpolation
#     .transform(lambda group: group.interpolate(method='linear', limit_direction='both').ffill())
# )
df_full[band_cols] = df_full.groupby(['id', 'crop', 'year'])[band_cols].transform(
    lambda group: group.interpolate(method='linear', limit_direction='both').ffill().bfill()
)

# Check if any NaNs remain after interpolation (this means there was no data to interpolate from)
remaining_nans_after_interp = df_full[band_cols].isnull().any(axis=1).sum()
if remaining_nans_after_interp > 0:
    print(f"\nWarning: {remaining_nans_after_interp} rows still contain NaNs in band columns after interpolation. These will result in NaNs in the wide format if an entire ID's data is missing.")


df_full['month'] = df_full['month'].astype(int).apply(lambda x: f"{x:02d}")

df_melted = df_full.melt(id_vars=['id', 'crop', 'year', 'month'], value_vars=band_cols,
                         var_name='band', value_name='value')

df_melted['band_month'] = df_melted['band'] + '_' + df_melted['month']

all_original_id_crop = df[['id', 'crop']].drop_duplicates().reset_index(drop=True)

df_wide_temp = df_melted.pivot_table(index=['id', 'crop', 'year'], columns='band_month', values='value').reset_index()

df_wide = pd.merge(all_original_id_crop, df_wide_temp.drop(columns=['year']), on=['id', 'crop'], how='left')


print(f"\nUnique IDs in final df_wide: {df_wide['id'].nunique()}")
print(f"Shape of final df_wide: {df_wide.shape}")
df_wide.head()


Unique IDs in final df_wide: 953
Shape of final df_wide: (953, 146)


,id,crop,B11_01,B11_02,B11_03,B11_04,B11_05,B11_06,B11_07,B11_08,...,B9_03,B9_04,B9_05,B9_06,B9_07,B9_08,B9_09,B9_10,B9_11,B9_12
0,0bCYpY,Cocoa,3045.683968,3454.167243,2925.771626,3201.394079,3480.353710,3759.313341,4038.272972,3610.377803,...,1420.314494,1415.896963,1475.649494,1535.402025,1595.154556,1501.586185,1408.017814,1314.449443,1607.773164,1754.499423
1,0DNGt7,Cocoa,2936.652057,3983.464052,2860.835448,3527.093041,4140.206844,4753.320646,5366.434448,4742.342048,...,1394.325644,1483.047674,1619.643086,1756.238498,1892.833910,1753.450596,1614.067282,1474.683968,2122.964629,1730.970780
2,0vYqno,Cocoa,2875.745483,3097.601692,2868.606690,3154.728181,2937.514033,2720.299885,2503.085736,2690.786236,...,1439.312957,1452.344098,1423.640779,1394.937460,1366.234141,1382.889786,1399.545431,1416.201077,1970.750096,1775.145329
3,1AYcTD,Cocoa,3078.060361,3452.227605,3161.562092,3240.053825,3152.375881,3064.697937,2977.019992,2725.732795,...,1458.624375,1470.251826,1465.836729,1461.421633,1457.006536,1392.453159,1327.899782,1263.346405,2500.526336,1707.145329
4,1dqP0R,Cocoa,2896.001885,3147.686275,2647.351433,3980.684389,4450.039090,4919.393791,5388.748492,4732.160131,...,1343.609729,1506.339367,1642.874686,1779.410005,1915.945324,1761.296506,1606.647687,1451.998869,1603.104827,1703.127828


In [18]:
df_wide.isna().sum()

id        0
crop      0
B11_01    0
B11_02    0
B11_03    0
         ..
B9_08     0
B9_09     0
B9_10     0
B9_11     0
B9_12     0
Length: 146, dtype: int64

In [17]:
df_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Columns: 146 entries, id to B9_12
dtypes: float64(144), object(2)
memory usage: 1.1+ MB




`Method Used:` Filling remaining NaN values with the mean of the respective band_month feature, calculated specifically for each crop type (df_wide.groupby('crop')[col].transform('mean')).

`Rationale:` If an (id, crop, year) combination has no available band data, the most logical imputation is to use information from other similar entities. The assumption here is that while individual crop locations have unique characteristics, the overall average spectral behavior for a given crop type (e.g., 'Cocoa' vs. 'Rubber') is a strong, crop-specific indicator of its typical reflectance values across different months.

`Aim to Maintain True Values:` This step is vital for maintaining the integrity of the crop class information, which is our target variable. Instead of using a global average (which would blend the spectral characteristics of all crops), we impute with the mean derived from its own crop group. This ensures that the imputed values reflect the typical spectral signature of that specific crop during that particular month. This is crucial for enabling the machine learning model to accurately distinguish between different crop types based on their characteristic spectral profiles. It directly addresses the idea that the "true" missing value for a cocoa farm should be closer to what other cocoa farms exhibit, rather than a general average of all crop types.

In [ ]:
# band_month_cols = [col for col in df_wide.columns if col not in ['id', 'crop']]

# # Fill NaNs using the mean of each column, grouped by 'crop'
# for col in band_month_cols:
#     crop_means = df_wide.groupby('crop')[col].transform('mean')

#     df_wide[col] = df_wide[col].fillna(crop_means)

In [19]:
df_wide.to_csv("FULL_TRAIN.csv")

### `Test Data`

In [21]:
with rasterio.open('test/s2_Unknown_ID_000167_2024_01.tif') as src:
    band_descriptions = [src.descriptions[i] if src.descriptions[i] else f'band_{i+1}' for i in range(src.count)]
    bands = [src.read(i + 1) for i in range(src.count)]
    transform = src.transform
    nodata = src.nodata

rows, cols = bands[0].shape
xs, ys = np.meshgrid(np.arange(cols), np.arange(rows))
xs, ys = rasterio.transform.xy(transform, ys, xs)
xs_flat = np.array(xs).flatten()
ys_flat = np.array(ys).flatten()

data = {'x': xs_flat, 'y': ys_flat}
for i, band in enumerate(bands):
    col_name = band_descriptions[i]
    data[col_name] = band.flatten()

# Create DataFrame
test = pd.DataFrame(data)

In [22]:
test.head()

,x,y,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,band_10,band_11,band_12
0,-7.473489,4.797318,2586,2291,2162,1873,2189,3308,3827,3749,4244,1433,3040,1952
1,-7.473399,4.797318,2585,2299,2152,1885,2182,3382,3924,3767,4261,1452,3015,1922
2,-7.473309,4.797318,2585,2280,2145,1877,2182,3382,3924,3707,4261,1452,3015,1922
3,-7.473220,4.797318,2585,2303,2168,1890,2181,3468,4034,3724,4484,1452,3012,1910
4,-7.473130,4.797318,2585,2303,2156,1867,2181,3468,4034,3974,4484,1452,3012,1910


In [13]:
test_path = './test'

data_test = []

pattern = re.compile(r"s2_(\w+)_ID_([a-zA-Z0-9]+)_(\d{4})_(\d{2})\.tif")

for filename in os.listdir(test_path):
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        match = pattern.match(filename)
        if match:
            crop, crop_id, year, month = match.groups()
            file_path = os.path.join(test_path, filename)

            with rasterio.open(file_path) as src:
                record = {
                    'crop': crop,
                    'id': crop_id,
                    'year': int(year),
                    'month': int(month)
                }

                band_names = src.descriptions  

                for i in range(1, src.count + 1):
                    band = src.read(i)
                    mask = band != src.nodata if src.nodata is not None else np.ones_like(band, dtype=bool)
                    mean_val = np.mean(band[mask])

                    band_name = band_names[i - 1] if band_names and band_names[i - 1] else f'band_{i}'
                    record[band_name] = mean_val

                data_test.append(record)

df_test = pd.DataFrame(data_test)

print(f"Unique IDs found in original TIFF files: {df_test['id'].nunique()}")



Unique IDs found in original TIFF files: 282


In [14]:
# Count unique months per ID
month_counts = df_test.groupby('id')['month'].nunique().sort_values(ascending=False).reset_index()
month_counts.columns = ['id', 'num_months']

# Filter IDs with fewer than 12 months
incomplete_ids = month_counts[month_counts['num_months'] < 12]

# Display the result
print(incomplete_ids)

# Count how many IDs have less than 12 months
print(f"Number of IDs with less than 12 months: {len(incomplete_ids)}")


         id  num_months
12   733943          11
13   628578          11
14   039466          11
15   752139          11
16   116131          10
..      ...         ...
277  873895           7
278  877892           7
279  882551           7
280  885113           7
281  000167           7

[270 rows x 2 columns]
Number of IDs with less than 12 months: 270


In [20]:
df_test.head()

,crop,id,year,month,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,band_10,band_11,band_12
0,Unknown,000167,2024,1,2609.715879,2319.438293,2164.212995,1925.427143,2148.793156,3227.035371,3749.058055,3572.120338,4075.692426,1413.550173,2855.518262,1886.657824
1,Unknown,000167,2024,2,2879.517878,2623.550557,2415.243368,2150.736640,2289.144175,3166.971165,3617.379854,3418.487505,3847.446751,1331.888889,2539.895425,1768.923875
2,Unknown,000167,2024,3,2498.327951,2198.668589,2122.976547,1806.127259,2180.645905,3973.881200,4830.557093,4538.913110,5271.803153,1412.000769,3303.197616,2003.978854
3,Unknown,000167,2024,4,2412.658593,2092.028066,2016.951942,1650.023453,2074.529412,4015.772011,4936.122261,4618.902730,5407.549020,1397.169166,3175.590158,1849.454441
4,Unknown,000167,2024,7,3982.745098,3830.524798,3743.940408,3867.745098,3970.679739,4742.793156,5225.122261,4903.528643,5513.909650,1891.722030,4415.630527,3571.981930


In [23]:
df_test.rename(columns={
    'band_1': 'B1',
    'band_2': 'B2',
    'band_3': 'B3',
    'band_4': 'B4',
    'band_5': 'B5',
    'band_6': 'B6',
    'band_7': 'B7',
    'band_8': 'B8',
    'band_9': 'B8A',
    'band_10': 'B9',
    'band_11': 'B11',
    'band_12': 'B12',
    
}, inplace=True)

In [24]:
# Ensure month is integer
df_test['month'] = df_test['month'].astype(int)

# List of all months
all_months = list(range(1, 13))

# Unique (id, crop, year) combinations
id_info = df_test[['id', 'crop', 'year']].drop_duplicates()

# Create full month grid for each id
full_index = (
    id_info
    .merge(pd.DataFrame({'month': all_months}), how='cross')  # all combinations
    .sort_values(['id', 'month'])
)

# Merge to get full data with NaNs for missing months
df_full = full_index.merge(df_test, on=['id', 'crop', 'year', 'month'], how='left')

# Interpolate band columns per ID
band_cols = ['B1','B2', 'B3', 'B4', 'B5','B6','B7','B8','B8A','B9', 'B11', 'B12']
df_full[band_cols] = (
    df_full 
    .sort_values(['id', 'month'])
    .groupby('id')[band_cols]
    .transform(lambda group: group.interpolate(method='linear', limit_direction='both'))
)

# Optional: check that now all IDs have 12 months
assert df_full.groupby('id')['month'].nunique().eq(12).all()

# Preview result
df_full


,id,crop,year,month,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12
0,000167,Unknown,2024,1,2609.715879,2319.438293,2164.212995,1925.427143,2148.793156,3227.035371,3749.058055,3572.120338,4075.692426,1413.550173,2855.518262,1886.657824
1,000167,Unknown,2024,2,2879.517878,2623.550557,2415.243368,2150.736640,2289.144175,3166.971165,3617.379854,3418.487505,3847.446751,1331.888889,2539.895425,1768.923875
2,000167,Unknown,2024,3,2498.327951,2198.668589,2122.976547,1806.127259,2180.645905,3973.881200,4830.557093,4538.913110,5271.803153,1412.000769,3303.197616,2003.978854
3,000167,Unknown,2024,4,2412.658593,2092.028066,2016.951942,1650.023453,2074.529412,4015.772011,4936.122261,4618.902730,5407.549020,1397.169166,3175.590158,1849.454441
4,000167,Unknown,2024,5,2936.020761,2671.526977,2592.614764,2389.264001,2706.579521,4258.112393,5032.455594,4713.778034,5443.002563,1562.020120,3588.936947,2423.630270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3379,992245,Unknown,2024,8,4027.089581,3881.139305,3764.057542,3794.908753,3866.300269,4753.803024,5269.646546,4939.355632,5553.103165,1706.081763,4170.993849,3233.670127
3380,992245,Unknown,2024,9,3800.832757,3564.128284,3428.487248,3358.893631,3460.398693,4518.314623,5075.836345,4732.798155,5400.257465,1534.974369,4015.079969,3081.981930
3381,992245,Unknown,2024,10,3574.575932,3247.117263,3092.916955,2922.878508,3054.497116,4282.826221,4882.026144,4526.240677,5247.411765,1363.866974,3859.166090,2930.293733
3382,992245,Unknown,2024,11,2341.537870,2044.500192,1881.590927,1606.885429,1806.259131,2823.266052,3335.464052,3174.748174,3575.893887,1337.672818,2290.131488,1537.816993


In [26]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3384 entries, 0 to 3383
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      3384 non-null   object 
 1   crop    3384 non-null   object 
 2   year    3384 non-null   int64  
 3   month   3384 non-null   object 
 4   B1      3384 non-null   float64
 5   B2      3384 non-null   float64
 6   B3      3384 non-null   float64
 7   B4      3384 non-null   float64
 8   B5      3384 non-null   float64
 9   B6      3384 non-null   float64
 10  B7      3384 non-null   float64
 11  B8      3384 non-null   float64
 12  B8A     3384 non-null   float64
 13  B9      3384 non-null   float64
 14  B11     3384 non-null   float64
 15  B12     3384 non-null   float64
dtypes: float64(12), int64(1), object(3)
memory usage: 423.1+ KB


In [27]:
# Ensure month is integer
df_test['month'] = df_test['month'].astype(int)

# List of all months
all_months = list(range(1, 13))

id_info = df_test[['id', 'crop', 'year']].drop_duplicates().reset_index(drop=True)

# Create full month grid for each id
full_index = (
    id_info
    .merge(pd.DataFrame({'month': all_months}), how='cross')
    .sort_values(['id', 'month'])
)

# Merge to get full data with NaNs for missing months
df_full_test = full_index.merge(df_test, on=['id', 'crop', 'year', 'month'], how='left')

# Interpolate band columns per ID
band_cols = ['B1','B2', 'B3', 'B4', 'B5','B6','B7','B8','B8A','B9', 'B11', 'B12']
# df_full[band_cols] = (
#     df_full
#     .sort_values(['id', 'month'])
#     .groupby(['id', 'crop', 'year'])[band_cols] # Group also by year and crop for interpolation
#     .transform(lambda group: group.interpolate(method='linear', limit_direction='both').ffill())
# )
df_full_test[band_cols] = df_full_test.groupby(['id', 'crop', 'year'])[band_cols].transform(
    lambda group: group.interpolate(method='linear', limit_direction='both').ffill().bfill()
)

# Check if any NaNs remain after interpolation (this means there was no data to interpolate from)
remaining_nans_after_interp2 = df_full_test[band_cols].isnull().any(axis=1).sum()
if remaining_nans_after_interp2 > 0:
    print(f"\nWarning: {remaining_nans_after_interp2} rows still contain NaNs in band columns after interpolation. These will result in NaNs in the wide format if an entire ID's data is missing.")


df_full_test['month'] = df_full_test['month'].astype(int).apply(lambda x: f"{x:02d}")

df_melted_test = df_full_test.melt(id_vars=['id', 'crop', 'year', 'month'], value_vars=band_cols,
                         var_name='band', value_name='value')

df_melted_test['band_month'] = df_melted_test['band'] + '_' + df_melted_test['month']

original_id_crop = df_test[['id', 'crop']].drop_duplicates().reset_index(drop=True)

df_wide_temp_test = df_melted_test.pivot_table(index=['id', 'crop', 'year'], columns='band_month', values='value').reset_index()

df_wide_test = pd.merge(original_id_crop, df_wide_temp_test.drop(columns=['year']), on=['id', 'crop'], how='left')


print(f"\nUnique IDs in final df_wide: {df_wide_test['id'].nunique()}")
print(f"Shape of final df_wide: {df_wide_test.shape}")
df_wide_test.head()


Unique IDs in final df_wide: 282
Shape of final df_wide: (282, 146)


,id,crop,B11_01,B11_02,B11_03,B11_04,B11_05,B11_06,B11_07,B11_08,...,B9_03,B9_04,B9_05,B9_06,B9_07,B9_08,B9_09,B9_10,B9_11,B9_12
0,000167,Unknown,2855.518262,2539.895425,3303.197616,3175.590158,3588.936947,4002.283737,4415.630527,4157.893599,...,1412.000769,1397.169166,1562.020120,1726.871075,1891.722030,1800.224529,1708.727028,1617.229527,1525.732026,1433.654364
1,004157,Unknown,2662.200308,4933.703191,2139.427528,5799.198770,5570.772908,5342.347046,5113.921184,5441.409458,...,1200.608997,3750.434448,3108.773420,2467.112393,1825.451365,1872.845060,1998.124567,2539.268743,1397.011150,1540.015763
2,010554,Unknown,2756.365244,4314.802768,3244.572857,3036.407920,3565.362168,3356.917339,3148.472511,3164.958478,...,1377.769319,1327.434833,1414.018454,1424.456747,1434.895040,1489.712514,1544.529988,1599.347463,1654.164937,1487.832757
3,016218,Unknown,3455.379854,2893.176086,3085.573626,4198.510573,4777.418429,5356.326285,5935.234141,5820.849385,...,1381.695502,2598.375240,2547.621428,2496.867615,2446.113802,2423.083333,2400.052864,2377.022395,2353.991926,1590.660131
4,018928,Unknown,2913.115725,3465.507882,3811.594387,2981.822760,3014.810842,3047.798923,3080.787005,3365.504293,...,1556.661669,1460.910419,1477.846341,1494.782263,1511.718185,1525.646290,1539.574394,1553.502499,1462.891196,1742.511726


In [28]:
df_wide_test.to_csv("FULL_TEST.csv", index=False)